In [1]:
!pip install openpyxl
import json
import csv
import requests
!pip install openpyxl
import pandas as pd
import sqlite3

In [2]:
Product = pd.read_excel('Product.xlsx')
Promotion = pd.read_excel('Promotion.xlsx')
Sales = pd.read_excel('Sales Fact.xlsx')
Time = pd.read_excel('Time.xlsx')
Store = pd.read_excel('Store.xlsx')


In [3]:
Sales.head()

,time_key,product_key,promotion_key,store_key,dollar_sales,unit_sales,dollar_cost,customer_count
0,1,1,1,15,78.35,58,81.19,38
1,1,2,1,10,114.89,85,115.88,48
2,1,3,1,7,112.21,83,123.76,77
3,1,4,1,12,49.55,37,50.56,35
4,1,5,11,19,27.26,56,24.50,45


In [4]:
#Load Data in sql lite

!pip install sqlalchemy
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///database.db')
Product.to_sql(name='Product', con=engine, if_exists='replace')
Promotion.to_sql(name='Promotion', con=engine, if_exists='replace')
Sales.to_sql(name='Sales', con=engine, if_exists='replace')
Time.to_sql(name='Time', con=engine, if_exists='replace')
Store.to_sql(name='Store', con=engine, if_exists='replace')

20

In [13]:
#quest 1
# How many sales did the company have?
pd.read_sql("select sum(unit_sales) as Total_Sales from Sales",engine)



,Total_Sales
0,550720


In [6]:
#quest 2
#How many products did we sale in a store last month?
pd.read_sql("SELECT st.store_number, SUM(unit_sales) AS Products_sold_in_last_month_Store_wise FROM Sales sa JOIN Store st ON sa.store_key = st.store_key JOIN Time T ON T.time_key = sa.time_key WHERE T.date BETWEEN '1995-12-01' AND '1995-12-31' group by st.store_number order by 1 ",engine)


,store_number,Products_sold_in_last_month_Store_wise
0,1,5112
1,2,4055
2,3,3881
3,4,5052
4,5,5037
5,6,3641
6,7,4205
7,8,4915
8,9,3947
9,10,5158


In [7]:
#quest 3
#Which one of our stores have the highest amount of sells
#answer : 12
pd.read_sql("SELECT store_number, SUM(unit_sales) AS Sales_in_a_Store FROM Sales sa JOIN Store st ON sa.store_key = st.store_key group by st.store_number order by 2 desc  ",engine)

,store_number,Sales_in_a_Store
0,12,29192
1,10,29045
2,3,28866
3,8,28618
4,1,28472
5,11,28132
6,19,27918
7,6,27762
8,9,27722
9,17,27596


In [8]:
#ques 4
#Which products are the most lucrative? 
#Answer : Buffalo Jerky

pd.read_sql("SELECT description as Lucrative_Products , sum(dollar_sales) - sum(dollar_cost) as profit FROM Sales sa JOIN Product pr ON pr.product_key = sa.product_key GROUP BY pr.description order by 2 desc;",engine)

,Lucrative_Products,profit
0,Buffalo Jerky,11022.52
1,Chicken Dinner,7809.59
2,Turkey Dinner,7262.20
3,Beef Stew,6971.91
4,Lasagna,6850.49
5,Paper Towels,6609.52
6,Dry Tissues,6411.96
7,Wet Wipes,6385.83
8,Clear Refresher,3371.03
9,Athletic Drink,3362.59


In [9]:
#quest 5
#What was the most lucrative day, month, or year
pd.read_sql("SELECT t.year as Lucrative_Year, sum(sa.dollar_sales) - sum(sa.dollar_cost) as profit FROM Sales sa JOIN Time t ON t.time_key = sa.time_key GROUP BY t.year order by profit desc",engine)


,Lucrative_Year,profit
0,1994,54282.79
1,1995,42318.26


In [22]:
#quest 6
#How many products are bought by our top 10% of customers?

pd.read_sql("select * from Sales order by customer_count desc FETCH FIRST 10 rows ONLY",engine)

OperationalError: (sqlite3.OperationalError) near "FETCH": syntax error
[SQL: select * from Sales order by customer_count desc FETCH FIRST 10 rows ONLY]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [28]:

pd.read_sql("select  ",engine)

,customer_count
0,0
1,1
2,2
3,3
4,4
...,...
96,96
97,97
98,98
99,99


In [11]:
#quest 6
#Who are our top 10% best sellers/employee last year?

In [29]:
pd.read_sql("select * from Promotion",engine)





,index,promotion_key,promotion_name,price_reduction_type,ad_type,display_type,coupon_type,ad_media_type,display_provider,promo_cost,promo_begin_date,promo_end_date
0,0,1,Blue Ribbon Discounts,Temporary,Daily Paper,None,None,Paper,None,2000,1994-10-01 00:00:00.000000,1994-10-15 00:00:00.000000
1,1,2,Red Carpet Closeout,Markdown,Sunday Paper,None,None,Paper,None,1000,1995-10-01 00:00:00.000000,1995-10-15 00:00:00.000000
2,2,3,Ad Blitz,None,Paper and Radio,None,None,Paper and Radio,None,7000,1994-11-10 00:00:00.000000,1994-11-17 00:00:00.000000
3,3,4,Ads and Racks,None,Paper and Radio,Rack,None,Paper and Radio,Smith,3000,1995-11-11 00:00:00.000000,1995-11-18 00:00:00.000000
4,4,5,Shelf Talkers,None,None,Shelf Tag,None,None,Jones,500,1994-10-01 00:00:00.000000,1994-12-31 00:00:00.000000
5,5,6,POS Grabbers,None,None,Register,None,None,Smith,600,1995-10-01 00:00:00.000000,1995-12-31 00:00:00.000000
6,6,7,Two for One,Coupon Only,Sunday Paper,None,Paper,Paper,None,1000,1994-11-15 00:00:00.000000,1994-11-30 00:00:00.000000
7,7,8,Cents Off Coupon,Coupon Only,Mailer,None,Mailer,None,None,1000,1995-11-15 00:00:00.000000,1995-11-30 00:00:00.000000
8,8,9,Redemption,Redemption Only,Attached,None,Attached,None,None,5000,1995-12-01 00:00:00.000000,1995-12-31 00:00:00.000000
9,9,10,Big Promo,Temporary,Daily Paper,End Aisle,None,Paper,Smith,8000,1995-12-15 00:00:00.000000,1995-12-31 00:00:00.000000
